# Hugging Face 추론 엔드포인트 예제

**[Hugging Face 추론 엔드포인트](https://ui.endpoints.huggingface.co/)**는 프로덕션에서 사용할 머신 러닝 모델을 배포하는 쉽고 안전한 방법을 제공합니다. 추론 엔드포인트는 개발자와 데이터 과학자 모두 인프라를 관리하지 않고도 AI 애플리케이션을 만들 수 있도록 지원합니다. 자동 확장을 통한 대량 요청 처리, 제로 스케일링을 통한 인프라 비용 절감, 고급 보안 제공 등 배포 프로세스를 몇 번의 클릭으로 단순화합니다.

추론 엔드포인트는 다음에서 시작할 수 있습니다. https://ui.endpoints.huggingface.co/


이 예제에서는 대화형 모델(예: `https://huggingface.co/meta-llama/Llama-2-13b-chat-hf`)에 대한 실행 중인 엔드포인트가 있다고 가정합니다.

### 1. easyllm 라이브러리 가져오기

In [ ]:
# 필요한 경우 OpenAI Python 라이브러리의 최신 버전으로 설치 및/또는 업그레이드합니다.
%pip install --upgrade easyllm 

### 2. 채팅 API 호출 예제

추론에 엔드포인트를 사용하려고 하므로 `model` 매개변수를 정의할 필요가 없습니다. `easyllm.clients.huggingface`를 가져오기 전에 환경 변수 `HUGGINGFACE_API_BASE`를 노출하거나 `huggingface.api_base` 값을 덮어써야 합니다.

채팅 API 호출에는 두 가지 필수 입력만 있습니다.
- `messages`: 메시지 개체 목록이며 각 개체에는 두 가지 필수 필드가 있습니다.
    - `role`: 메신저의 역할( `system`, `user` 또는 `assistant`)
    - `content`: 메시지 내용(예: `아름다운 시를 써주세요`)

채팅 형식이 실제로 어떻게 작동하는지 확인하기 위해 예제 채팅 API 호출을 살펴보겠습니다.

In [1]:
from easyllm.clients import huggingface

# 여기서는 기본값을 덮어씁니다. 환경 변수를 사용할 수도 있습니다.
huggingface.prompt_builder = "llama2"
huggingface.api_base = "YOUR_ENDPOINT_URL"

# 이 모듈은 환경 변수 HUGGINGFACE_TOKEN 또는 HuggingFace CLI 구성 파일에서 HuggingFace API 키를 자동으로 로드합니다.
# huggingface.api_key="hf_xxx"

response = huggingface.ChatCompletion.create(
    messages=[
        {"role": "system", "content": "\n당신은 도움이 되고 정중하며 정직한 조수입니다. 항상 안전하면서 가능한 한 도움이 되도록 답변하십시오. 답변에는 유해하거나 비윤리적이거나 인종 차별적이거나 성 차별적이거나 유독하거나 위험하거나 불법적인 내용이 포함되어서는 안 됩니다. 답변이 사회적으로 편향되지 않고 긍정적인지 확인하십시오.\n\n질문이 이해가 되지 않거나 사실적으로 일관성이 없는 경우 올바르지 않은 내용을 답변하는 대신 이유를 설명하십시오. 질문에 대한 답변을 모르는 경우 거짓 정보를 공유하지 마십시오."},
        {"role": "user", "content": "똑똑."},
        {"role": "assistant", "content": "누구세요?"},
        {"role": "user", "content": "사과."},
    ],
      temperature=0.9,
      top_p=0.6,
      max_tokens=1024,
)
response

{'id': 'hf-0lL5H_yyRR',
 'object': 'chat.completion',
 'created': 1691096023,
 'choices': [{'index': 0,
   'message': {'role': 'assistant', 'content': ' 사과 누구?'},
   'finish_reason': 'eos_token'}],
 'usage': {'prompt_tokens': 149, 'completion_tokens': 5, 'total_tokens': 154}}

보시다시피 응답 개체에는 몇 가지 필드가 있습니다.
- `id`: 요청 ID
- `object`: 반환된 개체 유형(예: `chat.completion`)
- `created`: 요청 타임스탬프
- `model`: 응답을 생성하는 데 사용된 모델의 전체 이름
- `usage`: 프롬프트, 완성 및 합계를 계산하여 회신을 생성하는 데 사용된 토큰 수
- `choices`: 완성 개체 목록(`n`을 1보다 크게 설정하지 않은 경우 하나만)
    - `message`: 모델에서 생성한 메시지 개체( `role` 및 `content` 포함)
    - `finish_reason`: 모델이 텍스트 생성을 중지한 이유(`stop` 또는 `max_tokens` 제한에 도달한 경우 `length`)
    - `index`: 선택 항목 목록에서 완성의 인덱스

다음을 사용하여 회신만 추출합니다.

In [2]:
print(response['choices'][0]['message']['content'])

 사과 누구?


## 채팅 완료 요청을 스트리밍하는 방법

사용자 지정 엔드포인트를 만들어 모델에 채팅 완료 요청을 스트리밍할 수 있습니다.

In [6]:
from easyllm.clients import huggingface

huggingface.prompt_builder = "llama2"

# 여기에서 엔드포인트 URL을 덮어쓸 수 있으며 localhost:8000일 수도 있습니다.
huggingface.api_base = "YOUR_ENDPOINT_URL"

# ChatCompletion 요청
response = huggingface.ChatCompletion.create(
    messages=[
        {'role': 'user', 'content': "10까지 세어보세요."}
    ],
    stream=True  # 이번에는 stream=True로 설정합니다.
)

for chunk in response:
    delta = chunk['choices'][0]['delta']
    if "content" in delta:
        print(delta["content"],end="")

  물론입니다! 여기 있습니다:

1. 하나
2. 둘
3. 셋
4. 넷
5. 다섯
6. 여섯
7. 일곱
8. 여덟
9. 아홉
10. 열!